# LIBRARY BLOCK

In [2]:
from selenium import webdriver
import json
import time
from time import sleep
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import pandas as pd

# SET UP BLOCK

In [3]:
NUMBER_OF_YEAR = 12

# FUNCTION BLOCK

In [4]:
def year_choosing(quy=False):
    if quy:
        return
    year_button = driver.find_element(By.CSS_SELECTOR, 'input[onchange="redirectFromSelect2(this.value,0)"]')
    year_button.click()

    sleep(2)

In [5]:
def year_monitoring(move_type, times=1):
    if move_type == "before":
        for _ in range(times):
            year_before = driver.find_element(By.CSS_SELECTOR, 'img[alt="Xem dữ liệu trước"]')
            year_before.click()
            sleep(2)
    elif move_type == "after":
        for _ in range(times):
            year_after = driver.find_element(By.CSS_SELECTOR, 'img[alt="Xem dữ liệu tiếp"]')
            year_after.click()
            sleep(2)
    else:
        raise ValueError("move_type must be either 'before' or 'after'")
    sleep(2)

In [6]:
def get_table_data(ma,business_performance_results_table=False, balance_sheet=False, cash_flow_statement=False,quy=False):
    result = {}
    df_result = pd.DataFrame(result)
    if business_performance_results_table == True:
        numbers = [f'0{i}' if i < 10 else str(i) for i in range(1, 61)]
        to_balance_sheet = driver.find_element(By.CSS_SELECTOR, 'a[id*="aNhom2"]')
        to_balance_sheet.click()
        sleep(2)

    elif balance_sheet == True:
        numbers = [i for i in range(100,441)]
        to_balance_sheet = driver.find_element(By.CSS_SELECTOR, 'a[id*="aNhom1"]')
        to_balance_sheet.click()
        sleep(2)

    elif cash_flow_statement == True:    
        numbers = [f'0{i}' if i < 10 else str(i) for i in range(1, 71)]
        
    else:
        return
    for number in numbers:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, f'tr[class*=r_item][id="{number}"] td')
            if elements == []:
                continue
        except:
            continue
        element_name = re.sub(r'^.*?\.', '', elements[0].text)
        element_name = re.sub(r'[^A-Za-zÀ-ỹ ]+', '', element_name).strip()
        elements_values = [element.text for element in elements[1:5]]
        result[f"{element_name}"] = elements_values
        
        element_time = driver.find_elements(By.CSS_SELECTOR, 'div[id="divTableHeader"] td')[1:5]
        if quy:
            quarters = [re.search(r'Quý\s*(\d)', x.text).group(1) for x in element_time]
            result["quy"] = quarters
            years = [re.search(r'(\d{4})', x.text).group(1) for x in element_time]
            result["years"] = years
        else:
            result["quy"] = 0
            result["years"] = [year.text for year in element_time]

        
        df_result = pd.DataFrame(result)
        df_result = df_result.replace('', None).apply(lambda col: col.str.replace(',', '').astype(float) if col.dtype == 'object' else col)
        df_result["ma"] = ma
        
    return df_result



# CODE EXECUTION BLOCK

In [7]:
global driver
driver = webdriver.Chrome()
driver.get("https://cafef.vn/du-lieu/bao-cao-tai-chinh/HPG/incsta/2024/4/2/0/bao-cao-tai-chinh-.chn")

In [8]:


# Danh sách mã cổ phiếu
# df = pd.read_json('data/danh_sach_ma_simple.json', orient='values')
# codes_list = df.values.flatten().tolist()
# codes_list = [code.upper() for code in codes_list]

top_5_market_cap = ['HPG', 'STB', 'TCB', 'MBB', 'SSI']
codes_list = top_5_market_cap

df_business_performance, df_balance_sheet = pd.DataFrame(), pd.DataFrame()
for code in codes_list[1:]:
    driver.get(f"https://cafef.vn/du-lieu/bao-cao-tai-chinh/{code}/incsta/2024/4/0/0/0/bao-cao-tai-chinh-.chn")
    sleep(2)

    year_choosing(quy=True)

    number_of_year_list = [4] * (NUMBER_OF_YEAR // 4) + ([NUMBER_OF_YEAR % 4] if NUMBER_OF_YEAR % 4 != 0 else [])
    for idx,num_year in enumerate(number_of_year_list):
        if idx != 0:
            year_monitoring("before", times=num_year)

        business_performance_data = get_table_data(ma=code, business_performance_results_table=True, quy=True)
        df_business_performance = business_performance_data if df_business_performance.empty else pd.concat([df_business_performance, business_performance_data], axis=0).drop_duplicates(subset=['quy','years', 'ma'], keep='last')

        balance_sheet_data = get_table_data(ma=code, balance_sheet=True, quy=True)
        df_balance_sheet = balance_sheet_data if df_balance_sheet.empty else pd.concat([df_balance_sheet, balance_sheet_data], axis=0).drop_duplicates(subset=['quy','years', 'ma'], keep='last')
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")


df_business_performance.to_csv(f'data/finance_data/business_performance_{current_time}.csv', index=False)


df_balance_sheet.to_csv(f'data/finance_data/balance_sheet_{current_time}.csv', index=False)

display(df_business_performance)
display(df_balance_sheet)


print("Dữ liệu của tất cả các công ty đã được lưu vào sheet 'TongHop' trong file 'du_lieu_2018_2023.xlsx'")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[id*="aNhom2"]"}
  (Session info: chrome=143.0.7499.42); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6fea98895
	0x7ff6fea988f0
	0x7ff6fe87165d
	0x7ff6fe8c9a33
	0x7ff6fe8c9d3c
	0x7ff6fe91df67
	0x7ff6fe91ac97
	0x7ff6fe8bac29
	0x7ff6fe8bba93
	0x7ff6fedb05f0
	0x7ff6fedaaf30
	0x7ff6fedc9696
	0x7ff6feab5d94
	0x7ff6feabed3c
	0x7ff6feaa1fb4
	0x7ff6feaa2165
	0x7ff6fea87e92
	0x7fffc22ee8d7
	0x7fffc366c53c
